Your objective is to build a series of scatter plots to showcase the following relationships:

- Temperature (F) vs. Latitude
- Humidity (%) vs. Latitude
- Cloudiness (%) vs. Latitude
- Wind Speed (mph) vs. Latitude

Your final notebook must:

Randomly select at least 500 unique (non-repeat) cities based on latitude and longitude.
Perform a weather check on each of the cities using a series of successive API calls.
Include a print log of each city as it's being processed with the city number, city name, and requested URL.
Save both a CSV of all data retrieved and png images for each scatter plot.
As final considerations:

You must use the Matplotlib and Seaborn libraries.
You must include a written description of three observable trends based on the data.
You must use proper labeling of your plots, including aspects like: Plot Titles (with date of analysis) and Axes Labels.
You must include an exported markdown version of your Notebook called  README.md in your GitHub repository.
See Example Solution for a reference on expected format.

In [107]:
# -----------------------------------------------------------------------------------
# Step 1: Import the modules 
# -----------------------------------------------------------------------------------
import seaborn as sns
import json
import matplotlib as plt
import requests as req
import pandas as pd
import random as rd
from citipy import citipy
from localenv import apiKey

In [114]:
# -----------------------------------------------------------------------------------
# Step 2: Generate a city list without duplicates using citipy
# -----------------------------------------------------------------------------------

# create counter
i = 0

# create city list
city_list = []

while i < 500:
    # set random lat and long
    latitude = rd.uniform(-90.0,90.0)
    longitude = rd.uniform(-180.0,180.0)

    # try to pull in a random value
    city = [citipy.nearest_city(latitude,longitude).city_name, citipy.nearest_city(latitude,longitude).country_code]
    
    if city_list.count(city) == 0:
        city_list.append(city)
        i += 1

In [115]:
# -----------------------------------------------------------------------------------
# Step 2: Store all our basic API data. I'd use params, but I couldn't get it to work
# with the unique structure of some of these parameters
# -----------------------------------------------------------------------------------

tempUnits = 'Imperial'
openWeatherURL = "http://api.openweathermap.org/data/2.5/weather?"

Your objective is to build a series of scatter plots to showcase the following relationships:

- Temperature (F) vs. Latitude
- Humidity (%) vs. Latitude
- Cloudiness (%) vs. Latitude
- Wind Speed (mph) vs. Latitude


In [116]:
# -----------------------------------------------------------------------------------
# Step 3: Pull data based on city IDs 
# -----------------------------------------------------------------------------------
# create a list that we'll store all our data in
weatherData_list = []


# loop through all the cities, pull data and add to dictionaries, and print requested log information
for i in range(len(city_list)):
    # set value for current city and country code so we don't have to keep calling for it
    currentCityName = city_list[i][0]
    currentCountryCode = city_list[i][1]

    # get response
    response = req.get(f"{openWeatherURL}q={currentCityName},{currentCountryCode}&units={tempUnits}&APPID={apiKey}").json()
    
    # print out information as it's being processed
    print(f"City #{i+1}")
    print(f"City URL: http://api.openweathermap.org/data/2.5/weather?id={currentCityID}")
    print(f"City Name: {response['name']}")
    print(f"City ID: {response['id']}")
    print("----------------------------------------------------------------------------")    
    
    # add information from response to list
    weatherData_list.append({'ID':response['id'],'Name': response['name'],'Latitude':response['coord']['lat'],\
                           'Humidity (%)':response['main']['humidity'],\
                           'Temperature (Fahrenheit)':response['main']['temp'],\
                           'Wind Speed':response['wind']['speed'],'Cloudiness':response['clouds']['all'] })
    

City #1
City URL: http://api.openweathermap.org/data/2.5/weather?id=1307641


KeyError: 'name'

In [105]:
# -----------------------------------------------------------------------------------
# Step 4: Create a pretty dataframe that we can reference because visual aids are
# the actual best thing ever
# -----------------------------------------------------------------------------------

# create dataframe
cleanedWeather_df = pd.DataFrame(weatherData_list).set_index('ID')

# rearrange columns sensibly
cleanedWeather_df = cleanedWeather_df[['Name', 'Latitude', 'Temperature (Fahrenheit)',\
                                       'Humidity (%)', 'Wind Speed','Cloudiness']]
cleanedWeather_df.head()

,Name,Latitude,Temperature (Fahrenheit),Humidity (%),Wind Speed,Cloudiness
ID,,,,,,
1493162,Salym,60.06,0.21,64,8.16,56
1252783,Yarada,17.65,80.13,100,11.07,0
5848280,Kapaa,22.08,72.43,88,19.46,90
4032243,Vaini,-21.20,80.60,88,8.39,75
1797543,Tianpeng,28.88,66.72,97,6.60,92


In [101]:
# -----------------------------------------------------------------------------------
# Step 5: Generate charts.
#  1) Temperature (F) vs. Latitude
#  2) Humidity (%) vs. Latitude
#  3) Cloudiness (%) vs. Latitude
#  4) Wind Speed (mph) vs. Latitude
# -----------------------------------------------------------------------------------

# Chart 1: Temperature vs Latitude
